<a href="https://colab.research.google.com/github/SUMIN-WEE/NLP/blob/master/0709_popcorn(LSTM)_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# popcorn data를 LSTM으로 감성분석
# (+loss, acc)

In [ ]:
# 교재 코드 lstm 분석

# 교재 문장(161p)_vocabulary 만들기(keras_tokenizer) (O)
# vocab의 index로 표현(text to sequence. 인덱스 벡터)
# review[0] -> [404,70...] => one-hot (리뷰 하나가 2D로 변경)
## 트리 구조 : 단어 개수 한 리뷰당 (174, vocab_size) 의 트리 구조

# 길이가 다 다르기 때문에 길이를 맞춤(keras_padding ->174개로)
## lstm_many to one : 2차원 -> 1차원 벡터로 나옴 -> ffn(받기. 2d 라 받을 수 있음)


## 리뷰 1개_ 
## embedding layer 에 집어넣음 -> 각각의 리뷰들을 one-hot으로 변경 
## -> 32개의 embedding size에 넣어줌 -> output : 32개의 벡터 나옴
# 길이가 다 다르기 때문에 길이를 맞춤(keras_padding)
## lstm, cnn_ 3차원 데이터 사용 가능
## lstm_many to one : 2차원 -> 1차원 벡터로 나옴 -> ffn(받기. 2d 라 받을 수 있음)
# '어떤 길이 사용?' -> eda 결과 참조, 174개로 padding (맞춰줌)

# 교재 -172p부분 

In [ ]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import nltk
from nltk.stem import LancasterStemmer
import pickle
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

nltk.download('punkt')
nltk.download('stopwords')

# Commented out IPython magic to ensure Python compatibility.
# 학습 데이터를 읽어온다.
# %cd '/content/drive/My Drive/Colab Notebooks'

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/popcorn_data/labeledTrainData.tsv', header=0, sep='\t', quoting=3)
df['review'][0]

In [ ]:
df['sentiment'].head

In [ ]:
# Pre-processing
stemmer = LancasterStemmer()
stopwords = nltk.corpus.stopwords.words('english')

processed_text = []
for review in df['review']:
    # 1. 영문자와 숫자만 사용한다. 그 이외의 문자는 공백 문자로 대체한다.
    review = review.replace('<br />', ' ')   # <br> --> space
    review = re.sub("[^a-zA-Z]", " ", review)    # 영문자만 사용

    tmp = []
    for word in nltk.word_tokenize(review):
        # 2. 길이가 2 이하인 단어와 stopword는 제거한다.
        if len(word) > 2 and word not in stopwords:
            # 3. Lemmatize
            tmp.append(stemmer.stem(word.lower()))
    processed_text.append(' '.join(tmp))

In [ ]:
# Pre-processing
stemmer = LancasterStemmer()
stopwords = nltk.corpus.stopwords.words('english')

processed_text = []
for review in df['review']:
    # 1. 영문자와 숫자만 사용한다. 그 이외의 문자는 공백 문자로 대체한다.
    review = review.replace('<br />', ' ')   # <br> --> space
    review = re.sub("[^a-zA-Z]", " ", review)    # 영문자만 사용

    tmp = []
    for word in nltk.word_tokenize(review):
        # 2. 길이가 2 이하인 단어와 stopword는 제거한다.
        if len(word) > 2 and word not in stopwords:
            # 3. Lemmatize
            tmp.append(stemmer.stem(word.lower()))
    processed_text.append(' '.join(tmp))

In [ ]:
# 학습 데이터
X_train, X_test, y_train, y_test = train_test_split(processed_text, list(df['sentiment']), test_size = 0.3)

# 학습 데이터를 저장해 둔다.
with open('/content/drive/MyDrive/Colab Notebooks/data/popcorn_data/popcorn.pkl', 'wb') as f:
    pickle.dump([X_train, y_train, X_test, y_test], f, pickle.DEFAULT_PROTOCOL)

In [ ]:
# 학습 데이터
X_train, X_test, y_train, y_test = train_test_split(processed_text, list(df['sentiment']), test_size = 0.3)

# 학습 데이터를 저장해 둔다.
with open('/content/drive/MyDrive/Colab Notebooks/data/popcorn_data/popcorn.pkl', 'wb') as f:
    pickle.dump([X_train, y_train, X_test, y_test], f, pickle.DEFAULT_PROTOCOL)

In [ ]:
fig,axe = plt.subplots(ncols=1)
fig.set_size_inches(6,3)
sns.countplot(df['sentiment'])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score
import pickle

# Commented out IPython magic to ensure Python compatibility.
# %cd '/content/drive/My Drive/Colab Notebooks'

# 학습 데이터를 읽어온다.
with open('/content/drive/MyDrive/Colab Notebooks/data/popcorn_data/popcorn.pkl', 'rb') as f:
    X_train, y_train, X_test, y_test = pickle.load(f)

In [ ]:
# 학습
pipe = Pipeline([('tf_vect', TfidfVectorizer(max_df=50, max_features=10000)),
                  ('lr_clf', LogisticRegression(max_iter = 500, C = 10))])

pipe.fit(X_train, y_train)
pred = pipe.predict(X_test)
pred_probs = pipe.predict_proba(X_test)[:, 1]

print('정확도 =', accuracy_score(y_test, pred), ' ROC-AUC =', roc_auc_score(y_test, pred_probs))

In [ ]:
len(pipe['tf_vect'].vocabulary_)

In [ ]:
# padding

from bs4 import BeautifulSoup 
from nltk.corpus import stopwords
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer

# def preprocessing(review, remove_stopwords = False):
#   # html 태그 제거

review = df['review'][0]
review_text = BeautifulSoup(review, "html5lib").get_text()

stop_words = set(stopwords.words('english'))

review_text = review_text.lower()
words = review_text.split()
words = [w for w in words if not w in stop_words]

In [ ]:
clean_review = ' '.join(words)
print(clean_review)

In [ ]:
def preprocessing(review, remove_stopwords = False):
  # 대문자->소문자
  words = review_text.split()

  if remove_stopwords:
    # 불용어 제거
    stops = set(stopwords.words("english"))
    words = [w for w in words if not w in stop_words]
    clean_review = ' '.join(words)

  else:
    clean_review = ''.join(words)

  return(clean_review)

In [ ]:
clean_train_reviews = []
for review in df['review']:
  clean_train_reviews.append(preprocessing(review, remove_stopwords = True))

len(clean_train_reviews)

In [ ]:
clean_train_df = pd.DataFrame({'review' : clean_train_reviews, 'sentiment': df['sentiment']})

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_reviews)
text_sequences = tokenizer.texts_to_sequences(clean_train_reviews)

In [ ]:
word_vocab = tokenizer.word_index
word_vocab["<PAD>"]=0
print(word_vocab)

In [ ]:
print(len(word_vocab))

In [ ]:
pipe = Pipeline([('tf_vect', TfidfVectorizer(max_df=50, max_features=10000)),
                  ('lr_clf', LogisticRegression(max_iter = 500, C = 10))])

pipe.fit(X_train, y_train)
pred = pipe.predict(X_test)
pred_probs = pipe.predict_proba(X_test)[:, 1]

In [ ]:
# LSTM (+ embedded layer)_이후로는 코드 진행 못함

In [ ]:
embedding_vector_length = 32
model = Sequential([
    Embedding(input_dim=top_words, # 5000
              output_dim=embedding_vector_length, # 32
              input_length=max_review_length), 
    LSTM(50), # 원래는 100, 
    Dropout(0.5), 
    Dense(25, activation='sigmoid'), 
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=64)
print("training complete")

scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {:.2f}".format(scores[1]*100))

In [ ]:
# 답

In [ ]:
# Word Embedding & LSTM을 이용한 감성분석
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import pickle

# Commented out IPython magic to ensure Python compatibility.
# %cd '/content/drive/My Drive/Colab Notebooks'

# 학습, 시험 데이터를 읽어온다.
with open('data/popcorn.pkl', 'rb') as f:
    xd_train, yd_train, xd_test, yd_test = pickle.load(f)

xd_train[0]

# 학습, 시험 데이터를 합쳐서 사전 (vocabulary)을 구축한다.
x_data = xd_train + xd_test
y_target = np.array(yd_train + yd_test).reshape(-1,1)

# 사전을 구축하고 학습 데이터를 워드 인덱스로 표현한다.
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_data)
x_idx = tokenizer.texts_to_sequences(x_data)

# vocabulary
word2idx = tokenizer.word_index
word2idx['<PAD>'] = 0

# 각 리뷰의 길이를 측정하고, 평균 길이를 구한다.
x_len = [len(x) for x in x_idx]

print('평균 =', np.mean(x_len))
print('최소 =', np.min(x_len))
print('최대 =', np.max(x_len))
print('중앙값 =', np.median(x_len))

# 길이 분포를 확인한다.
plt.hist(x_len, bins=50)
plt.show()

# 각 리뷰의 길이를 max_seq_len로 맞춘다. 길면 자르고, 짧으면 padding을 추가한다.
MAX_SEQ_LEN = 200
x_review = pad_sequences(x_idx, maxlen=MAX_SEQ_LEN, padding='post', truncating='post')
x_review[5000]

# 학습 데이터와 시험 데이터로 분리한다.
x_train, x_test, y_train, y_test = train_test_split(x_review, y_target, test_size=0.2)

# Embedding & LSTM 모델을 생성한다.
vocab_size = len(word2idx)
EMBEDDING_DIM = 32
HIDDEN_DIM = 64

x_input = Input(batch_shape=(None, x_train.shape[1]))
e_layer = Embedding(input_dim=vocab_size, output_dim=EMBEDDING_DIM)(x_input)
e_layer = Dropout(rate=0.5)(e_layer)
r_layer = LSTM(HIDDEN_DIM, dropout=0.5)(e_layer)
y_output = Dense(1, activation='sigmoid')(r_layer)

model = Model(x_input, y_output)
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0005))
model.summary()

# 학습
hist = model.fit(x_train, y_train, validation_data = (x_test, y_test), batch_size = 1024, epochs = 30)

# Loss history를 그린다
plt.plot(hist.history['loss'], label='Train loss')
plt.plot(hist.history['val_loss'], label = 'Test loss')
plt.legend()
plt.title("Loss history")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.show()

# 시험 데이터로 학습 성능을 평가한다
pred = model.predict(x_test)
y_pred = np.where(pred > 0.5, 1, 0)
accuracy = (y_pred == y_test).mean()
print("\nAccuracy = %.2f %s" % (accuracy * 100, '%'))